## Import and prep CTpet and PET data

script copied from "CT and PET DICOM load and resample"

In [ ]:
import dicom
import dicom_numpy
import os
import numpy as np
from numpy import flip
import nibabel as nib
from matplotlib import pyplot as plt, cm
from matplotlib import image
import math
from scipy.ndimage import zoom
import SimpleITK as sitk
import re
import datetime

In [ ]:
baseCT = "D:/CNNdata/Bowel_Segmentation/RATHL_Data_Validation/CT"
basePET = "D:/CNNdata/Bowel_Segmentation/RATHL_Data_Validation/PT"
ID = 11

In [ ]:
PathDicomCT = os.path.join(str(baseCT) + str(ID))

In [ ]:
# function to extract voxel data from DICOM to Numpy using the dicom_numpy module
# seems to take into account the rescale intercept appropriately

def extract_voxel_data(list_of_dicom_files):
    datasets = [dicom.read_file(f) for f in list_of_dicom_files]
    try:
        voxel_ndarray, ijk_to_xyz = dicom_numpy.combine_slices(datasets)
    except dicom_numpy.DicomImportException as e:
        # invalid DICOM data
        raise
    return voxel_ndarray

In [ ]:
## create the list of DICOM files in the CT and PET folders

lstFilesDCM_CT = []  # create an empty list
for dirName, subdirList, fileList in os.walk(PathDicomCT):
    for filename in fileList:
        if ".dcm" in filename.lower():  # check whether the file's DICOM
            lstFilesDCM_CT.append(os.path.join(dirName,filename))

PathDicomPET = os.path.join(str(basePET) + str(ID))
lstFilesDCM_PET = []  # create an empty list
for dirName, subdirList, fileList in os.walk(PathDicomPET):
    for filename in fileList:
        if ".dcm" in filename.lower():  # check whether the file's DICOM
            lstFilesDCM_PET.append(os.path.join(dirName,filename))    

In [ ]:
# Get header info from CT
RefCT = dicom.read_file(lstFilesDCM_CT[0])

# Load dimensions based on the number of rows, columns, and slices (along the Z axis)
ConstPixelDims_CT = (int(RefCT.Rows), int(RefCT.Columns), len(lstFilesDCM_CT))

# Load spacing values (in mm)
ConstPixelSpacing = (float(RefCT.PixelSpacing[0]), float(RefCT.PixelSpacing[1]), float(RefCT.SliceThickness))
CTReconDiameter = round((RefCT.PixelSpacing[0]) * (RefCT.Rows))

In [ ]:
# This is needed later to find the slice number of the first slice of BB
# The array is sized based on 'ConstPixelDims'
a_CT = np.zeros(ConstPixelDims_CT[2])
i=0
# loop through all the DICOM files, reading the slice numbers to array
for filenameDCM in lstFilesDCM_CT:
    dsCT = dicom.read_file(filenameDCM)
    sliceNumber = dsCT.ImagePositionPatient[2]
    a_CT[i] = sliceNumber
    i=i+1
    
b_CT = np.sort(a_CT, axis=-1, kind='quicksort', order=None) #element 0 is inf-most slice
#b_CT

In [ ]:
# BBstartslice = b_CT[(Patients[ID][1])]
# print("BBstartslice = ", BBstartslice)

In [ ]:
CTarray = extract_voxel_data(lstFilesDCM_CT)

In [ ]:
PETarray = extract_voxel_data(lstFilesDCM_PET)

In [ ]:
#some CT data has a ring of -3000 values around the FOV circle ... not sure why
CTarray[CTarray < -1000] = -1000

In [ ]:
np.shape(CTarray)

In [ ]:
s = 99
fig = plt.figure(figsize=(6, 6))
im1 = plt.imshow(CTarray[:,:,s], vmin = -200, vmax = 500, cmap='gray')
im1 = plt.clim(-200,300)
plt.show()

In [ ]:
# Get header info from PET
RefPET = dicom.read_file(lstFilesDCM_PET[0])
# Load dimensions based on the number of rows, columns, and slices (along the Z axis)
ConstPixelDims_PET = (int(RefPET.Rows), int(RefPET.Columns), len(lstFilesDCM_PET))
# Load spacing values (in mm)
ConstPixelSpacing_PET = (float(RefPET.PixelSpacing[0]), float(RefPET.PixelSpacing[1]), float(RefPET.SliceThickness))

PETReconDiameter = round((RefPET.PixelSpacing[0]) * (RefPET.Rows))
#PETReconDiameter

In [ ]:
CTarrayExpand = np.expand_dims(CTarray, axis = 0) # adds a dimension
CTarrayTrans = np.transpose(CTarrayExpand, (3,0,1,2)) #numbers indicate where the original dimension should go
CTarrayNew = np.rot90(CTarrayTrans,1,(2,3))

In [ ]:
s = 99
fig = plt.figure(figsize=(6, 6))
im1 = plt.imshow(CTarrayNew[s,0,:,:], vmin = -200, vmax = 500, cmap='gray')
im1 = plt.clim(-200,300)
plt.show()

In [ ]:
PETarrayExpand = np.expand_dims(PETarray, axis = 0) # adds a dimension
PETarrayTrans = np.transpose(PETarrayExpand, (3,0,1,2)) #numbers indicate where the original dimension should go
PETarrayNew = np.rot90(PETarrayTrans,1,(2,3))

In [ ]:
np.shape(PETarrayNew)

# Resizing CT to match PET dimensions

In [ ]:
padextra = round((round((PETReconDiameter/CTReconDiameter)*ConstPixelDims_CT[0]) - ConstPixelDims_CT[0])/2)

In [ ]:
if padextra > 0:
    CTarrayPad = np.pad(CTarrayNew, ((0,0),(0,0),(padextra,padextra),(padextra,padextra)), mode = 'constant', constant_values = (-1000))
    print("padextra > 0")
elif padextra == 0:
    CTarrayPad = np.copy(CTarrayNew)
    print("padextra = 0")
else:
    CTarrayPad = CTarrayNew[:, :, -padextra:padextra, -padextra:padextra]
    print("padextra < 0")

In [ ]:
xyZoomFactor = (ConstPixelDims_PET[0]/ConstPixelDims_CT[0])*(CTReconDiameter/PETReconDiameter)

In [ ]:
zZoomFactor = np.size(PETarray,2)/np.size(CTarray,2)

In [ ]:
CTarrayNewResize = zoom(CTarrayPad, (zZoomFactor,1, xyZoomFactor, xyZoomFactor))

In [ ]:
padextra

In [ ]:
s = 99
fig = plt.figure(figsize=(6, 6))
im1 = plt.imshow(CTarrayPad[s,0,:,:], vmin = -200, vmax = 500, cmap='gray')
im1 = plt.clim(-200,300)
plt.show()

In [ ]:
for filenameDCM in lstFilesDCM_PET:
    dsPET = dicom.read_file(filenameDCM)
    ds_Units = dsPET[0x0054,0x1001].value
    ds_halfLife = dsPET[0x0054,0x0016][0][0x0018,0x1075].value 
    ds_PtWeight = dsPET.PatientWeight
    ds_PtName = dsPET[0x10,0x10].value
    ds_injectedDose = dsPET[0x0054,0x0016][0][0x0018,0x1074].value
    ds_seriesDate = dsPET[0x0008,0x0021].value
    ds_seriesTime = dsPET[0x0008,0x0031].value
    ds_acquisitionDate = dsPET[0x0008,0x0022].value
    ds_acquisitionTime = dsPET[0x0008,0x0032].value
    ds_scanTime = dsPET[0x0008,0x0031].value
    ds_startTime = dsPET[0x0054,0x0016][0][0x0018,0x1072].value
    ds_rescaleSlope = dsPET[0x0028,0x1053].value
    ds_rescaleIntercept = dsPET[0x0028,0x1052].value

In [ ]:
try:
    seriesTimeFormat  = datetime.datetime.strptime(ds_seriesTime, "%H%M%S")
except:
    seriesTimeFormat  = datetime.datetime.strptime(ds_seriesTime, "%H%M%S.%f")
    
try:
    acquisitionTimeFormat  = datetime.datetime.strptime(ds_acquisitionTime, "%H%M%S")
except:
    acquisitionTimeFormat  = datetime.datetime.strptime(ds_acquisitionTime, "%H%M%S.%f")

In [ ]:
try:
    startTimeFormat  = datetime.datetime.strptime(ds_startTime, "%H%M%S") # if the time is like this 105600
except:
    startTimeFormat  = datetime.datetime.strptime(ds_startTime, "%H%M%S.%f") # if the time is decimal like this 105600.00

try:
    scanTimeFormat  = datetime.datetime.strptime(ds_scanTime, "%H%M%S")
except:
    scanTimeFormat  = datetime.datetime.strptime(ds_scanTime, "%H%M%S.%f")
    
    
decaytime = (scanTimeFormat - startTimeFormat).total_seconds()

In [ ]:
DecayedDose = ds_injectedDose * pow(2,(-decaytime/ds_halfLife))

In [ ]:
SUVbwScaleFactor = (ds_PtWeight * 1000) / DecayedDose

In [ ]:
PETarrayNewSUVbw = ((PETarrayNew) + ds_rescaleIntercept) * SUVbwScaleFactor
#PETarrayNewSUVbw = ((PETarrayNew*ds_rescaleSlope) + ds_rescaleIntercept) * SUVbwScaleFactor # rescaleSlope not needed?

In [ ]:
np.shape(PETarrayNewSUVbw)

In [ ]:
s = 99
fig = plt.figure(figsize=(6, 6))
im1 = plt.imshow(CTarrayNewResize[s,0,:,:], vmin = -200, vmax = 500, cmap='gray')
im1 = plt.clim(-200,300)
im2 = plt.imshow(PETarrayNewSUVbw[s,0,:,:],alpha=.5) # transparency is alpha
im2 = plt.colorbar()
plt.show()

In [ ]:
np.max(PETarrayNewSUVbw[s,0,:,:])

In [ ]:
np.max(PETarrayNewSUVbw[99,0,:,:])